In [16]:
import pandas as pd
import numpy as np
import requests

In [51]:
url = "http://localhost:3000/api/graphql"
headers={"Content-Type": "application/json"}
body = {
    "query": """
    query Users {
      users {
        _id
        interactions {
          movieId
          rating
        }
      }
    }
    """
}

response = requests.post(url, json=body, headers=headers)
data=[]
try:
    data=response.json()['data']['users']
except ValueError:
    print("Error: Invalid JSON response. Status Code:", response.status_code)
    print("Response Text:", response.text)

In [67]:
#create a flattend list
flattened_data = []
for user in data:
    user_id=user['_id']
    for interaction in user['interactions']:
        flattened_data.append({
            'user_id':user_id,
            'movie_id':interaction['movieId'],
            'rating':interaction['rating']
        })
flattened_data
#convert into dataframe
data_frame=pd.DataFrame(flattened_data)
print(data_frame.columns)

Index(['user_id', 'movie_id', 'rating'], dtype='object')


In [70]:
#convert into user_movie matrix
user_movie_matrix = data_frame.pivot_table(index="user_id", columns="movie_id", values="rating",aggfunc='mean')
user_movie_matrix

movie_id,67b74ca00106f2047810825d,67b752290106f204781082d7,67b9de3b54720ab39e7545da,67cb271aa87f3b8f8f79726a,67cc43ca76bdb95f827c7782,67cc469b76bdb95f827c7b3f,67cc55be76bdb95f827c8113,67cf1f24ecd7c832508adc9c,67dd897329af0958f589563e,67e2f2a35f6cd300680a3708,67e8094541c70d18248a9ff6
user_id,,,,,,,,,,,
67b9d787ce46683627debdca,5.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
67cb02cf8c985a197d764adf,NaN,NaN,5.0,4.0,4.0,4.0,4.0,4.0,5.0,3.0,5.0
